In [1]:
import os

In [2]:
%pwd

"d:\\MLOp's\\END TO END1\\datascienceproject\\research"

In [3]:
os.chdir("../")
%pwd

"d:\\MLOp's\\END TO END1\\datascienceproject"

this changes our point of execution from this research folder to the main project folder

NOW WE WILL USE AND NDERSTAND THE CONCEPT AND USAGE OF DATACLASS

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
     root_dir: Path
     source_URL: str
     local_data_file: Path
     unzip_dir: Path

in normal class we always use self function also we are more focused on creating functions but in data class our primary focus is in creating data value pairs not creating any kind of functions

***STEP 5***: Updating the configuration manager in src->config

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        """all the paths are defined in the constants->init.py file """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

        """now we got all the file paths but now we need the most important info that is the data link folder link extractedfolder path now we'll import that"""
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """the -> DataIngestionConfig is used for type hinting i.e which datatype will be returned from the fucntion"""
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

***STEP:6*** downloading and extracting the aaip file basically

In [7]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [8]:
###component-data inestion

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
##downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"file already exists of path: {self.config.local_data_file}")

    ##extracting the cipfile 
    def extract_zip_file(self):
        """
        zip_file_path:str
        extracts the zip file into the data directory
        function returns none
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_file()

    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-11 11:17:14,924]:INFO:common:yaml file: config\config.yaml loaded successfully]
[2026-01-11 11:17:14,924]:INFO:common:yaml file: params.yaml loaded successfully]
[2026-01-11 11:17:14,930]:INFO:common:yaml file: schema.yaml loaded successfully]
[2026-01-11 11:17:14,932]:INFO:common:Created directory at: artifacts]
[2026-01-11 11:17:14,933]:INFO:common:Created directory at: artifacts/data_ingestion]
[2026-01-11 11:17:16,951]:INFO:2726705499:artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6840:7D7C8:39F28E:83B68C:69633962
Accept-Ranges: bytes
Date: Sun, 11 Jan 2026 05